In [2]:
!pip install -q kaggle


In [5]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"weapagawa","key":"f927d4bfe12545cfe8dfc83e2e941860"}'}

In [6]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import numpy
import re
from collections import Counter
import os
import csv
import pandas as pd
import random
import string
import math
trainingSenPath = '/content/drive/MyDrive/Colab_Notebooks/trainHPOST.words'
with open(trainingSenPath, encoding='utf8') as wordD:
    trainSent = [line.rstrip('\n') for line in wordD] #output: 1 line of sentence
    trainwordSplit = [i for trainword in trainSent for i in trainword.split()] #output: 1 word (ARRAY)
trainingTagPath = '/content/drive/MyDrive/Colab_Notebooks/trainHPOST.tags'
with open(trainingTagPath,encoding='utf-8') as wordT:
    trainTag = [line.rstrip('\n') for line in wordT] #output: 1 line of sentence
    tagTrainSplit = [i for ttSplit in trainTag for i in ttSplit.split()] #output: 1 TAG
testingSenPath = '/content/drive/MyDrive/Colab_Notebooks/testHPOST.words'
with open(testingSenPath,encoding='utf-8') as testW:
    senTest = [line.rstrip('\n') for line in testW] #output: 1 line of sentence
    wordTestSplit = [i for wtestSplit in senTest for i in wtestSplit.split()] #output: 1 word
testingTagPath = '/content/drive/MyDrive/Colab_Notebooks/testHPOST.tags'
with open(testingTagPath,encoding='utf-8') as testW:
    senTags = [line.rstrip('\n') for line in testW] #output: 1 line of sentence
    tagSenSplit = [i for wtestSplit in senTest for i in wtestSplit.split()] #output: 1 word

FileNotFoundError: ignored

In [ ]:
uniquetag = []
# traverse for all elements
for x in tagTrainSplit:
    if x not in uniquetag:
        uniquetag.append(x) #unique list of wordtest file

In [ ]:
tagcnt = []
for i in range(0, len(uniquetag)):
    x = []
    x.append(tagTrainSplit.count(uniquetag[i]))
    x.append(uniquetag[i])
    tagcnt.append(x) 

In [ ]:
def tagoccurence(tag):
    for tagcount in tagcnt:
        if tagcount[1] == tag:
            return(tagcount[0])
            break;

In [ ]:
count = 0
ptagWords = []

for i in trainSent:
    wordtag=""
    tag = trainTag[count]
    count=count + 1
    for indx,word in enumerate(i.split()):
        tagged=tag.split()
        wordtag = wordtag+ word+"/"+tagged[indx]+" "
    ptagWords.append(wordtag)

In [ ]:
#training table
def tagCombo(tag):
    tagComboArr = []
    for x in trainTag:
        x = x.split(" ")
        for index,z in enumerate(x):
            if z == tag:
                indexList = [index-2,index-1,index,index+1,index+2]
                comboTag = ""
                for indx in indexList:
                    if indx >= 0 and indx < len(x):
                        if(len(comboTag) == 0):
                            comboTag = x[indx]
                        else:
                            comboTag = comboTag+" "+x[indx]
                    else:
                        if(len(comboTag) == 0):
                            comboTag = "NULL"
                        else:
                            comboTag = comboTag+" "+"NULL"
                tagComboArr.append(comboTag)
    return(tagComboArr)   

In [ ]:
#sentence with ambiguity tag and possible tags
loadLimit = 10;
counter = 0;
# individual with only 6 possible tags sorted by tag occurences desc HIGHER -> LOWER
processedSentence = []
for sentence in senTest:
    counter = counter + 1;
    arrSen=[]
    words = sentence.split(" ")
    for word in words:
        temptags = []
        possibletags=[]
        for indx,enumeratedWord in enumerate(trainwordSplit):
            if word == enumeratedWord:
                    temptags.append(tagTrainSplit[indx])
        possibletagOccurences = dict(Counter(temptags))
        #sort by tagoccurences desc HIGHER -> LOWER
        possibletagOccurences = sorted(possibletagOccurences, key=possibletagOccurences.get, reverse=True)
#         print(possibletagOccurences)
        if len(possibletagOccurences) > 6:
            possibletags.extend(possibletagOccurences[0:6])
        else:
            possibletags.extend(possibletagOccurences)
        if(len(possibletags) > 0):
            tagsChk = possibletags.count(possibletags[0]) == len(possibletags)
            if(tagsChk):
                x = word,"non-ambiguous",possibletags
                arrSen.append(x)
            else:
                x = word,"ambiguous",possibletags
                arrSen.append(x)
                
        else:
            y = word,"unknown",possibletags
            arrSen.append(y)
   
    processedSentence.append(arrSen)  
    
    if(loadLimit == counter):
        break;

In [ ]:
print(processedSentence)

In [ ]:
#target creator
finalProccessedSentence = []

for sen in processedSentence:
  for index,currWord in enumerate(word):
    if(index == 0):
      if(currWord[1] == "ambiguous"):
        currentTag = ""
        currentTagCnt = 0
        for tag in currWord[2]:
          # initTag = ['NULL','NULL',tag]
          tagcombo = tagCombo(tag)
          cnt = 0
          for tagcombo in tagcombo:
            tagFreq = tagcombo.split(" ")
            # //counter if tags are the same as initTag
            if(tagFreq[0] == 'NULL' and tagFreq[1] == 'NULL' and tagFreq[2] == tag):
              cnt = cnt+1;
          if(cnt > currentTagCnt):
            currentTag = tag
            currentTagCnt = cnt
            x = currWord[0]+"/"+currentTag
        finalProccessedSentence.append(x)
    elif(currWord[1] == "non-ambiguous"):
      z = currWord[2][0]
      x = currWord[0]+"/"+z
      finalProccessedSentence.append(x)
    else:
      x = currWord[0]+"/"+"UNKNOWN"
      finalProccessedSentence.append(x)
#         elif(index == 1):
# #code here
#             currentTag = ""
#             currentTagFrequency = 0
#             if(currWord[1] == "ambiguous"):
#                 for tag in currWord[2]:
#                     initTag = ['NULL','NULL',tag]
#                     tagcombo = tagCombo(tag)
#                     cnt = 0
#                     for tagcombo in tagcombo:
#                         tagFreq = tagcombo.split(" ")
#                         if(tagFreq[0] == 'NULL' and tagFreq[1] == 'NULL' and tagFreq[2] == tag):
#                                 tagFreq[:3]
#                                 cnt = cnt+1;
#                 if(cnt > currentTagFrequency):
#                     currentTag = tag
#                     currentTagFrequency = cnt
#                     x = currWord[0]+"/"+currentTag
#                 finalProccessedSentence.append(x)
#             elif(currWord[1] == "non-ambiguous"):
#                 z = currWord[2][0]
#                 x = currWord[0]+"/"+z
#                 finalProccessedSentence.append(x)
#             else:
#                 x = currWord[0]+"/"+"UNKNOWN"
#                 finalProccessedSentence.append(x)
#             for tag in currWord[2]:
#                 print(tag)
#                 initTag = ['NULL','NULL',tag]
#                 tagcombo = tagCombo(tag)
#                 cnt = 0
#                 for tagcombo in tagcombo:
#                     tagFreq = tagcombo.split(" ")
#                     if(tagFreq[0] == 'NULL' and tagFreq[1] == 'NULL' and tagFreq[2] == tag):
#                             tagFreq[:3]
#                             cnt = cnt+1;
#                 if(cnt > currentTagFrequency):
#                     currentTag = tag
#                     currentTagFrequency = cnt
#             x = currWord[0]+"/"+currentTag
#             finalProccessedSentence.append(x)

In [ ]:
#first word per sentence/target tag
print(finalProccessedSentence)

In [ ]:
def starter(targetArr,possibletagArr):
    population_size = 100
    possibletags = possibletagArr
    target = targetArr

    class tagger:
        def __init__(self,tags):
            self.tagger = tags
            self.fitness = self.fitness()
        @classmethod
        def loc_tags(self):
            random_tags = [random.randint(0,len(possibletags)-1)]
            random_tags = random.choice(possibletags)
            return random_tags
        @classmethod
        def possible_tags(self):
            poss_tags = [self.loc_tags() for i in range(len(target))]
            return poss_tags

        def mutate(self,parent2):
            child = []
            for  i,j in zip(self.tagger,parent2.tagger):
                prob = random.random()

                if prob < 0.46:
                    child.append(i)
                elif prob < 0.90:
                    child.append(j)
                else:
                    child.append(self.loc_tags())
            return tagger(child)

        def fitness(self):
            fitness_score = 0
            for i,j in zip(self.tagger,target):
                if i!= j:
                    fitness_score = fitness_score +1
            return fitness_score

    population = []
    generation = 1
    found=False

    for i in range(population_size):
        actual_tags = tagger.possible_tags()
        obj_actualtags = tagger(actual_tags)
        population.append(obj_actualtags)

    while not found:
        population = sorted(population,key = lambda x:x.fitness)
#         for x in population:
#             resultrow = x.tagger,"fitness:",x.fitness,"generation:",generation
#             resultArray.append(resultrow)

        if population[0].fitness == 0:
            found = True
            return(population[0].tagger,"fitness:",population[0].fitness,"gen:",generation)
            break

        new_population = []
        new_population.extend(population[0:10])
        adjusted_size = population_size /2

        for i in range(len(new_population),population_size):
            parent1 = random.choice(population[:int(adjusted_size)])
            parent2 = random.choice(population[:int(adjusted_size)])
            new_population.append(parent1.mutate(parent2))

        population = new_population
        generation = generation + 1

In [ ]:
#STARTER(TARGET,POSSIBLE_TAGS)
starter(["JJC","DTC","JJCS_CCP"],uniquetag)